<a href="https://colab.research.google.com/github/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/introduction_to_information_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Introduction to Information Search

You might have come across the term Information Retrieval in the context of search engines: for example, Google famously started its business by providing a powerful search algorithm that kept improving over time. The search for information, however, is a basic need that you may face not only in the context of searching online: for instance, every time you search for the files on your computer, you also perform sort of information retrieval. In fact, the task predates digital era: before computers and Internet became a commodity, one
had to manually wade through paper copies of encyclopedias, books, documents, files and so on. Thanks to the technology, the algorithms these days help you do many of these tasks automatically.

##Setup

In [1]:
import os
import math
import random
import string

import nltk
from nltk import word_tokenize, WordNetLemmatizer, pos_tag
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.text import Text

from operator import itemgetter

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
%%shell

wget -qq https://github.com/ekochmar/Essential-NLP/raw/master/cisi.zip

unzip -qq cisi.zip

##Step 1: Understanding the task

---

**Scenario 1**:

Imagine that you have to perform the search in a collection of documents yourself, i.e. without the help of the machine. For example, you have a thousand printed out notes and minutes related to the meetings at work, and you
only need those that discuss the management meetings. How will you find all such documents? How will you identify the most relevant of these?

---

if you were tasked with this in actual life, you would go through the
documents one by one, identifying those that contain the key words (like `management` and `meetings`) and split all the documents into two piles: e.g. those documents that you should keep and look into further and those that you can discard because they do not answer your information need in learning more about the management meetings. This task is akin to filtering.

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/1.png?raw=1' width='800'/>

Now, there are a couple of points that we did not get to discuss before: imagine there are a hundred of documents in total and you can quickly skim through them to filter out the most irrelevant ones – those that do not even mention either “meetings” or “management”.

Luckily, these days we have computers and most documents are stored electronically. Computers can really help us speed the things up here.



---

**Scenario 2** (based on Scenario 1, but more technical!):

Imagine that you have to perform the search in a collection of documents, this time with the help of the machine. For
example, you have a thousand notes and minutes related to the meetings at work stored in electronic format, and
you only need those that discuss the management meetings.
- First, how will you find all such documents? In other words, how can you code the search algorithm and what
characteristics of the documents should the search be based on?
- Second, how will you identify the most relevant of these documents? In other words, how can you implement a
sorting algorithm to sort the documents in order of decreasing relevance?

---

It allows you to leverage the computational power of the machine, but the drill is the same as before: get the machine to identify the texts that have the keywords in them, and then sort the “keep” pile according to the relevance of the texts, starting with the most relevant for the user or yourself to look at.

Despite us saying just now that the procedure is similar to how the humans perform the task (as in Scenario 1), there are actually some steps involved in getting the machine identify the documents with the keywords in them and sorting by relevance that we are not explicitly mentioning here. 

For instance, we humans have the following abilities that we naturally possess but machines naturally lack:

- We know what represents a word, while a machine gets in a sequence of symbols and does not, by itself, have a notion of what a “word” is.
- We know which words are keywords: e.g., if we are interested in finding the
documents on management meetings, we will consider those containing “meeting”
and “management”, but also those containing “meetings” and potentially even
“manager” and “managerial”. The machine, on the other hand, does not know that
these words are related, similar, or basically different forms of the same word.
- We have an ability to focus on what matters: in fact, when reading texts we usually skim through them rather than pay equal attention to each word. For instance, when
reading a sentence “Last Friday the management committee had a meeting”, which
words do you pay more attention to? Which ones express the key idea of this
message? Think about it – and we will return to this question later. The machines, on the other hand, should be specifically “told” which words matter more.
- Finally, we also intuitively know how to judge what is more relevant. The machines can make relevance judgments, too, but unlike us humans they need to be “told” how to measure relevance in precise numbers.

That, in a nutshell, represents the basic steps in the search algorithm.


<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/2.png?raw=1' width='800'/>

In this notebook, you will learn about other NLP techniques to preselect words, map the different forms of the same word to each other and weigh the words according to how much information they contribute to the task. Then you will build an information search algorithm that for any query (for example, “management meetings”) will find the most relevant documents in the collection of documents (for example, all minutes of the past managerial meetings sorted by their relevance).

Suppose you have built such an application following all the steps. You type in a query and the algorithm returns a document or several documents that are supposedly relevant to this query. How can you tell whether the algorithm has picked out the right documents?

Let’s use a dataset of documents and queries, where the documents are labeled with respect to their relevance to the queries. You will use this dataset as your gold standard, and before using the information search algorithm in practice, evaluate its performance against the ground truth labels in the labeled dataset.




###Data and data structures

You are going to use a publicly available dataset labeled for the task. That
means, a dataset with a number of documents and various queries, and a labeled list specifying which queries correspond to which documents. Once you implement and evaluate a search algorithm on such data labeled with ground truth, you can apply it to your own documents in your own projects.

You will use the dataset collected by the Centre for Inventions and Scientific Information (CISI), which contains abstracts and some additional metadata on the journal articles on information systems and information retrieval.

You will need to keep precisely three data structures for this application: 
- one for the documents, 
- another one for the queries, and 
- the third one matching the queries to the documents

Information search is based on the idea that the content of a document or set of documents is relevant given the content of a particular query, so both documents and queries data structures should keep the contents of all documents and all queries. 

What would be the best way to keep track of which content represents which document?

The most informative and useful way would be to assign a unique identifier – an index – to each document and each query. You can imagine, for example, storing content of the documents and queries in two separate tables, with each row representing a single document or query, and row numbers corresponding to the documents and queries ids. In Python, tables can be represented with dictionaries.

Now, if you keep two Python dictionaries (tables) matching each unique document
identifier (called key) to the document’s content (called value) in documents dictionary and matching each unique query identifier to the query’s content in queries dictionary, how should you represent the relevance mappings? 

You can use a dictionary structure again: this time, the keys will contain the queries ids, while the values should keep the matching documents ids. Since each query may correspond to multiple documents, it would be best to
keep the ids of the matching documents as lists.

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/3.png?raw=1' width='800'/>

As this figure shows, query with id 1 matches documents with ids `1` and `1460`, therefore the mappings data structure keeps a list of `[1, 1460]` for query `1`; similarly it keeps `[3]` for query `2`, `[2]` for query `112`, and an empty list for query `3`, because in this example there are no
documents relevant for this query.

Now let’s look into the CISI dataset and code the data reading and initialization step. All documents are stored in a single text file CISI.ALL. It has a peculiar format: it keeps the abstract of each article and some additional information, such as the index in the set, the title, authors’ list and cross-references – a list of indexes for the articles that cite each other.

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/4.png?raw=1' width='800'/>

For the information search application, arguably the most useful information is the content of the abstract: abstracts in the articles typically serve as a concise summary of what the article presents, something akin to a snippet.

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/5.png?raw=1' width='800'/>

As you can see, the field identifiers such as .A or .W are separated from the actual text by new line. In addition, the text within each field, for example, the abstract may be spread across multiple lines. Ideally, we would like to convert this format into something like text.

Note that for the text that falls within the same field, e.g. .W, the line breaks (“\n”) are replaced with whitespaces, so each line now starts with a field identifier followed by the field content:

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/6.png?raw=1' width='800'/>

The format is much easier to work with: you can now read the text line by line,
extract the unique identifier for the article from the field .I, merge the content of the fields `.T`, `.A` and `.W`, and store the result in the documents dictionary as `{1: "18 Editions of the … this country and abroad."}`.

In [4]:
def read_documents():
  file = open("cisi/CISI.ALL")
  merged = ""

  for a_line in file.readlines():
    # Unless a string starts with a new field identifier, add the content to the current field separating the content from the previous line with a whitespace; 
    #votherwise, start a new line with the next identifier and field.
    if a_line.startswith("."):
      merged += "\n" + a_line.strip()
    else:
      merged += " " + a_line.strip()

    documents = {}
    content = ""
    doc_id = ""

    for a_line in merged.split("\n"):
      if a_line.startswith(".I"):
        doc_id = a_line.split(" ")[1].strip()  # doc_id can be extracted from the line with the .I field identifier
      elif a_line.startswith(".X"):
        documents[doc_id] = content
        content = ""
        doc_id = ""
      else:
        content += a_line.strip()[3:] + " "  # Otherwise, keep extracting the content from other fields (.T, .A and .W) removing the field identifiers themselves

  file.close()   
  return documents

As a sanity check, print out the size of the dictionary (make sure it contains all 1460 articles) and print out the content of the very first article.

In [5]:
documents = read_documents()
print(len(documents))
print(documents.get("1"))

1460
 18 Editions of the Dewey Decimal Classifications Comaromi, J.P. The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad. 


The queries are stored in `CISI.QRY` file and follow a very similar format: half the time, you see only two fields – `.I` for the unique identifier and `.W` for the content of the query. 

Other queries though are formulated not as questions but rather as abstracts from other articles. In such cases, the query also has an `.A` field for the authors’ list, `.T` for the title and `.B` field, which keeps the reference to the original journal in which the abstract was published.

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/7.png?raw=1' width='800'/>

We are going to only focus on the unique identifiers and the content of the query itself (fields `.W` and `.T`, where available), so the code below is quite similar to the above as it allows you to populate the queries dictionary with data:

In [6]:
def read_queries():
  file = open("cisi/CISI.QRY")
  merged = ""

  for a_line in file.readlines():
    # Unless a string starts with a new field identifier, add the content to the current field separating the content from the previous line with a whitespace; 
    #votherwise, start a new line with the next identifier and field.
    if a_line.startswith("."):
      merged += "\n" + a_line.strip()
    else:
      merged += " " + a_line.strip()

    queries = {}
    content = ""
    query_id = ""

    for a_line in merged.split("\n"):
      if a_line.startswith(".I"):
        if not content == "":
          queries[query_id] = content
          content = ""
          query_id = ""
        query_id = a_line.split(" ")[1].strip()  # query_id can be extracted from the line with the .I field identifier
      elif a_line.startswith(".W"):
        content += a_line.strip()[3:] + " "  # Otherwise, keep adding content to the content variable

    # The very last query is not followed by any next .I field, so the strategy from above won’t work –
    # you need to add the entry for the last query to the dictionary using this extra step
    queries[query_id] = content

  file.close()   
  return queries

In [7]:
queries = read_queries()

# Print out the length of the dictionary (it should contain 112 entries), and the content of the very first query
print(len(queries))
print(queries.get("1"))

112
What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles? 


Finally, let's read in the mapping between the queries and the documents – we'll keep these in the mappings data structure – with tuples where each query index (key) corresponds to the list of one or more document indices (value):

In [8]:
def read_mappings():
  file = open("cisi/CISI.REL")

  mappings = {}

  for a_line in file.readlines():
    voc = a_line.strip().split()
    key = voc[0].strip()
    current_value = voc[1].strip()  # The key (query id) is stored in the first column, while the document id is stored in the second column
    value = []
    """
    If the mappings dictionary already contains some document ids for the documents matching the given
    query, you need to update the existing list with the current value; otherwise just add current value to the new list
    """
    if key in mappings.keys():
      value = mappings.get(key)
    value.append(current_value)
    mappings[key] = value

  file.close()   
  return mappings

In [9]:
mappings = read_mappings()
print(len(mappings))
print(mappings.keys())
print(mappings.get("1"))

76
dict_keys(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '37', '39', '41', '42', '43', '44', '45', '46', '49', '50', '52', '54', '55', '56', '57', '58', '61', '62', '65', '66', '67', '69', '71', '76', '79', '81', '82', '84', '90', '92', '95', '96', '97', '98', '99', '100', '101', '102', '104', '109', '111'])
['28', '35', '38', '42', '43', '52', '65', '76', '86', '150', '189', '192', '193', '195', '215', '269', '291', '320', '429', '465', '466', '482', '483', '510', '524', '541', '576', '582', '589', '603', '650', '680', '711', '722', '726', '783', '813', '820', '868', '869', '894', '1162', '1164', '1195', '1196', '1281']


That’s it – you have successfully initialized one dictionary for documents with the ids linked to the articles content, another dictionary for queries linking queries ids to their correspondent texts, and the mappings dictionary, which matches the queries ids to the lists of relevant document ids.

Now, you are all set to start implementing the search algorithm for this data.

### Boolean search algorithm

Let’s start with the simplest approach: the information need is formulated as a query. If you extract the words from the query, you can then search for the documents that contain these words and return these documents, as they should be relevant to the query.

Here is the algorithm in a nutshell:
- Extract the words from the query
- For each document, compare the words in the document to the words in the query
- Return the document as relevant if any of the query words occurs in the document

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/8.png?raw=1' width='800'/>

The very first step in this algorithm is extraction of the words from both queries and documents. You may recall that text comes in as a sequence of symbols or characters, and the machine needs to be told what a word is – you used a special NLP tool called tokenizer to extract words.

In [10]:
def get_words(text):
  word_list = [word for word in word_tokenize(text.lower())]  # Text is converted to lower case and split into words
  return word_list

In [11]:
doc_words = {}
query_words = {}

# Entries in both documents and queries are represented as word lists
for doc_id in documents.keys():
  doc_words[doc_id] = get_words(documents.get(doc_id))
for qry_id in queries.keys():
  query_words[qry_id] = get_words(queries.get(qry_id))  

# check out the length of the dictionaries (these should be the same as before – 1460 and 112), 
# and check what words are extracted from the first document and the first query
print(len(doc_words))
print(doc_words.get("1"))
print(len(doc_words.get("1")))

print(len(query_words))
print(query_words.get("1"))
print(len(query_words.get("1")))

1460
['18', 'editions', 'of', 'the', 'dewey', 'decimal', 'classifications', 'comaromi', ',', 'j.p.', 'the', 'present', 'study', 'is', 'a', 'history', 'of', 'the', 'dewey', 'decimal', 'classification', '.', 'the', 'first', 'edition', 'of', 'the', 'ddc', 'was', 'published', 'in', '1876', ',', 'the', 'eighteenth', 'edition', 'in', '1971', ',', 'and', 'future', 'editions', 'will', 'continue', 'to', 'appear', 'as', 'needed', '.', 'in', 'spite', 'of', 'the', 'ddc', "'s", 'long', 'and', 'healthy', 'life', ',', 'however', ',', 'its', 'full', 'story', 'has', 'never', 'been', 'told', '.', 'there', 'have', 'been', 'biographies', 'of', 'dewey', 'that', 'briefly', 'describe', 'his', 'system', ',', 'but', 'this', 'is', 'the', 'first', 'attempt', 'to', 'provide', 'a', 'detailed', 'history', 'of', 'the', 'work', 'that', 'more', 'than', 'any', 'other', 'has', 'spurred', 'the', 'growth', 'of', 'librarianship', 'in', 'this', 'country', 'and', 'abroad', '.']
113
112
['what', 'problems', 'and', 'concerns',

Now let’s code the simple search algorithm. We will refer to it as the
Boolean search algorithm since it relies on presence (1) or absence (0) of the query words in the documents:

In [12]:
def retrieve_documents(doc_words, query):
  docs = []
  query_word = []
  for doc_id in doc_words.keys():
    found = False
    i = 0
    # Keep iterating through the words in the query word list until either of the two conditions is satisfied
    while i < len(query) and not found:
      word = query[i]
      if word in doc_words.get(doc_id):
        docs.append(doc_id)
        query_word.append(word)
        found = True
      else:
        i += 1
  return (docs, query_word)

In [13]:
# Check the results: select a query by its id (e.g., query with id 3 here), print out the ids of the documents
# that the algorithm found (e.g., the first 100, as there may be many),check how many there are in total
docs, query_word = retrieve_documents(doc_words, query_words.get("3"))
print(docs[:100])
print(len(docs))
print(query_word)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102']
1397
['is', 'is', 'is', 'information', 'is', 'is', '.', 'is', 'definitions', 'is', 'is', 'information', 'what', 'is', 'information', 'is', 'what', 'is', 'is', 'is', 'is', 'is', 'information', 'what', 'is', 'is', 'information', 'what', 'is', 'is', 'is', 'information', 'information', 'is', '.', '.', 'is', 'is', '.', 'is', '.', 'is', 'is', 'is', 'is', 'is', 'is', 'is', '.', 'is', 'is', 'information', 'inf

In [14]:
# Let’s, for example, look into how the algorithm decided on the documents relevant for query with id 6
docs, query_word = retrieve_documents(doc_words, query_words.get("6"))
print(docs[:100])
print(len(docs))
print(query_word)

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100']
1460
['there', 'are', 'are', 'for', 'for', 'are', 'for', 'there', 'for', 'are', 'are', 'for', 'what', 'are', 'communication', 'are', 'what', 'and', 'for', 'for', 'for', 'possibilities', 'are', 'what', 'are', 'are', 'for', 'what', 'are', 'are', 'are', 'are', 'between', 'are', 'there', ',', 'and', ',', 'between', 'for', 'for', 'for', 'are', 'for', 'are', 'there', 'are', 'are', 'are', 'are', 'are', 'for', '

As it shows, the query is matched to the document based on occurrence of such words as “there”, “this”, “the”, “and”, “is” and even a comma since punctuation marks are part of the word list returned by the tokenizer:

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/9.png?raw=1' width='800'/>

On the face of it, there is a considerable word overlap between the query and the document, yet if you read the text of the query and the text of the document, they don’t seem to have any ideas in common, so in fact this document is not relevant for the given query at all! 

It seems like the words on the basis of which the query and the document are matched here are simply the wrong ones – they are somewhat irrelevant to the actual information need expressed in the query. 

How can you make sure that the query and the documents are matched on the basis of more meaningful words?


---
**Exercise**

Another way to match the documents to the queries would be to make it a requirement that the document should contain all the words from the query rather than any.

Is this a better approach? Modify the code of the simple Boolean search algorithm to match documents to the queries on the basis of all words, and compare the results.

---

You will notice that it is rarely the case that a document, even if it is generally relevant,
contains all words from the query (at the very least, it does not have to contain question
words like “what” and “which” from the query to be relevant). Therefore, this more
conservative approach of returning only the documents with all query words in them will
work even worse at this stage – it simply will not find any relevant documents.



In [15]:
def retrieve_documents(doc_words, query):
  docs = []
  for doc_id in doc_words.keys():
    # here, you are interested in the documents that contain all words
    found = True
    i = 0
    # iterate through words in the query
    while i < len(query) and found:
      word = query[i]
      if not word in doc_words.get(doc_id):
        # if the word is not in document, turn found flag off and stop
        found = False
      else:
        # rwise, move on to the next query word
        i += 1

    # if all words are found in the document, the last index is len(query)-1 add the doc_id only in this case
    if i == len(query) - 1:
      docs.append(doc_id)
  return docs

In [16]:
docs = retrieve_documents(doc_words, query_words.get("112"))
print(docs[:100])
print(len(docs))

[]
0


In fact, it is a very rare case that you may have any single document that contains all the words from the query, therefore, with this approach, you will likely get no relevant documents returned for any queries in this dataset.


Before we move on, let’s summarize which steps of the algorithm you have implemented so far: you have read the data, initialized the data structures and tokenized the texts.

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/10.png?raw=1' width='800'/>

##Step 2: Preprocessing the data

Since we have identified several weaknesses of the current algorithm. 

Let’s look into further preprocessing steps that will help you represent the content of both the documents and the queries in a more informative way.

###Preselecting the words that matter: stopwords removal

The main problem with the search algorithm identified so far is that it considers all words in the queries and documents as equally important. This leads to poor search results, but on top of that it is also intuitively incorrect.

You may notice that not all words are equally meaningful in the sentences above. A good test for that would be to ask yourself whether you can define in one phrase what a particular word means: for example, what does “the” mean? You can say that “the” does not have a precise meaning of its own, rather it serves a particular function.

In linguistic terms, such words are called function words. You might even notice that when you read a text, for example an article or an email, you tend to skim over such words without paying much attention to them.

What happens to the search algorithm when these words are present? You have seen in the example before that they don’t help identify the relevant texts, so in fact the algorithm’s effort is wasted on them. What would happen if the less meaningful words were not taken into consideration?

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/11.png?raw=1' width='800'/>

You can see that, were the less meaningful words removed before matching documents to queries, document 1 would not stand a chance – there is simply not a single word overlapping between the query and this document. You can also see that the words that are not grayed out concisely summarize the main idea of the text.

**This suggests the first improvement to the developed algorithm: let’s remove the less meaningful words. In NLP applications, the less meaningful words are called stopwords,** and luckily you don’t have to bother with enumerating them – since stopwords are highly repetitive in English, most NLP toolkits have a specially defined stopwords list, so you can rely on this list when processing the data, unless you want to customize it.





In [17]:
def process(text):
  stoplist = set(stopwords.words("english"))  # use English stopwords
  # Tokenize text, convert it to lower case and only add the words if they are not included in the stoplist and are not punctuation marks
  word_list = [word for word in word_tokenize(text.lower())
               if not word in stoplist and not word in string.punctuation
              ]

  return word_list

In [18]:
# Check the result of these preprocessing steps on some documents or queries, e.g. document 1
word_list = process(documents.get("1"))
print(word_list)

['18', 'editions', 'dewey', 'decimal', 'classifications', 'comaromi', 'j.p.', 'present', 'study', 'history', 'dewey', 'decimal', 'classification', 'first', 'edition', 'ddc', 'published', '1876', 'eighteenth', 'edition', '1971', 'future', 'editions', 'continue', 'appear', 'needed', 'spite', 'ddc', "'s", 'long', 'healthy', 'life', 'however', 'full', 'story', 'never', 'told', 'biographies', 'dewey', 'briefly', 'describe', 'system', 'first', 'attempt', 'provide', 'detailed', 'history', 'work', 'spurred', 'growth', 'librarianship', 'country', 'abroad']


That is, the preprocessing step helps removing the stopwords like “of” and “the” from the word list.

###Matching forms of same word: morphological processing

One effect that stopwords and punctuation marks removal has is optimization of search algorithm – the words that do not matter much are removed, so the computational resources are not wasted on them. In general, the more concise and the more informative the data representation is, the better.

This brings us to the next issue. Take a look at the query with id 15
and document with id 27, which are a match according to the ground truth mappings:

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/11.png?raw=1' width='800'/>
> The words highlighted in blue will be matched between the query and the document; the ones in red will be missed.

The reason for this mismatch is that words may take different forms in different contexts: some contexts may require a mention of a single object or concept like “system”, while others may need multiple “systems” to be mentioned. 

Such different forms of a word that depend on the context and express different aspects of meaning, for instance multiplicity of “systems”, are technically called morphological forms, and when you see a word like “systems” and try to match it to its other variant “system” you are dealing with morphology.

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/12.png?raw=1' width='800'/>

Stemming takes word matching one step further and tries to map related words across the board, and this means not just the forms of the very same word. For that, the stemmers rely on a set of rules that try to reduce the related words to the same basic core.

The stem in `{retrieve, retrieves, retrieved, retrieving, retrieval}` is retriev. So here is the difference with the technique that you used before – stemming might result in non-words, as for example, you
won’t find a word like retriev in a dictionary. To provide you with a couple of other examples, the stem for `{expect, expects, expected, expecting, expectation, expectations}` is expect and the stem for `{continue, continuation, continuing}` is continu.

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/13.png?raw=1' width='800'/>

Note that the stemmer tries to identify which part of the word is shared between the different forms and related words, and returns this part as a stem by cutting off the differing word endings.

Now let’s implement the stemming preprocessing step using NLTK’s stemming
functionality.



In [19]:
def process(text):
  stoplist = set(stopwords.words("english"))  # use English stopwords
  st = LancasterStemmer() # Initialize the LancasterStemmer
  # Apply stemming to the preprocessed text
  word_list = [st.stem(word) for word in word_tokenize(text.lower())
               if not word in stoplist and not word in string.punctuation
              ]

  return word_list

In [20]:
word_list = process(documents.get("27"))
print(word_list)

word_list = process("organize, organizing, organizational, organ, organic, organizer")
print(word_list)

['cost', 'analys', 'sim', 'proc', 'evalu', 'larg', 'inform', 'system', 'bourn', 'c.p', 'ford', 'd.f', 'comput', 'program', 'writ', 'us', 'sim', 'several-year', 'op', 'inform', 'system', 'comput', 'estim', 'expect', 'op', 'cost', 'wel', 'amount', 'equip', 'personnel', 'requir', 'tim', 'period', 'program', 'us', 'analys', 'sev', 'larg', 'system', 'prov', 'us', 'research', 'tool', 'study', 'system', 'many', 'compon', 'interrel', 'op', 'equ', 'man', 'analys', 'would', 'extrem', 'cumbersom', 'tim', 'consum', 'perhap', 'ev', 'impract', 'pap', 'describ', 'program', 'show', 'exampl', 'result', 'sim', 'two', 'sev', 'suggest', 'design', 'spec', 'inform', 'system']
['org', 'org', 'org', 'org', 'org', 'org']


This example is used here rather as a warning about the way stemmers work: while they are useful in mapping related words to each other, sometimes they might produce an unexpected output and map unrelated words together. This happens because stemmers sometimes go too far in their attempt to establish the correspondences. As the stemmer blindly applies a rather
general set of rules to all examples, some of these rules overgeneralize.

So, technically, the last two rules should not be applied to map cases like organ –> organize, because the two words do not mean similar things, and it would be better for the applications like search algorithm to make the distinction between the two groups of words
`{organize, organizing, organizational, organizer}` and `{organ, organic}`. However, unfortunately, the stemmer algorithm does not take into account what words mean, so once in a while it makes mistakes and connects unrelated words.

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/14.png?raw=1' width='800'/>

Since in some cases the stemmer would map together words that are not closely related to each other, your search algorithm might consider documents talking about organic products somewhat relevant for the query that asks about organizational skills. This is something to keep in
mind; in general, because the queries are mapped to the relevant documents on the basis of more than one word from the query such incorrect mappings are usually outweighed by the relevance of other words.

Before we move on, let’s summarize which steps of the algorithm you have implemented so far: you have read the data, initialized the data structures and tokenized the texts, removed stopwords and applied the stemming preprocessing.

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/15.png?raw=1' width='800'/>

##Step 3: Information weighing

Another problem with the simple Boolean search algorithm is that it can only return a list of documents that contain some or all of the words from the
query, but it cannot tell which of the documents are more relevant. 

You’ve seen before that when you run the algorithm, for most queries it returns a huge number of documents. Without some measure of relevance and relevance ordering it would be physically impossible to look through all the documents returned this way. What could serve as such a measure?

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/16.png?raw=1' width='800'/>

Suppose you try to find documents most relevant to the given query. After stopwords and punctuation marks removal you end up with the query words – let’s call them keywords – consisting of `{much, information, retrieval, dissemination, systems, cost}`. Which of the two documents appears to be more relevant? 

Document `doc_x` does not only contain more keywords than `doc_y`, each keyword also occurs more times, so it would be reasonable to assume that `doc_x` is more relevant – given a choice between these two documents, you should start with `doc_x` if you want to find the answer to the query. 

How can we take the factors like more keywords and higher number of occurrences into account?

### Weighing words with term frequency

The first requirement, that you should take into account all keywords, suggests that you need to keep track of the words used in the queries and documents. The second requirement that the number of occurrences of each of the keywords matters, suggests that you need to count the number of occurrences rather than simply register presence or absence of a keyword.

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/18.png?raw=1' width='800'/>

You can achieve this by keeping the number of occurrences for the keywords in a
table, or translating this into a Python data structure you can use a dictionary that will allow you to keep track of which counts correspond to which keywords.

The correspondent Python dictionaries will be as follows:
```
Query={much:1, information:1, retrieval:1, dissemination:1, systems:1, cost:1}
Doc_x={much:0, information:2, retrieval:1, dissemination:2, systems:3, cost:1}
Doc_y={much:0, information:1, retrieval:1, dissemination:1, systems:2, cost:0}
```

This approach, based on calculating frequency of occurrence, corresponds to the well-known technique in Information Retrieval called term frequency (tf). It relies on the idea that the more frequently the word (term) is used in a document, the more relevant this document becomes to the query. We will use the word term instead of “word” from now on following this widely accepted convention: after all, since you apply stemming, not all keywords keep
to be proper “words” anymore (think of the case of retriev).




In [21]:
def get_terms(text):
  stoplist = set(stopwords.words("english"))
  terms = {}
  st = LancasterStemmer()
  word_list = [st.stem(word) for word in word_tokenize(text.lower()) if not word in stoplist and word not in string.punctuation]
  # Estimate the counts for each term and populate the dictionary
  for word in word_list:
    terms[word] = terms.get(word, 0) + 1
  
  return terms

In [22]:
doc_terms = {}
query_terms = {}

# Populate the term frequency dictionaries for all documents and all queries
for doc_id in documents.keys():
  doc_terms[doc_id] = get_terms(documents.get(doc_id))
for query_id in queries.keys():
  query_terms[query_id] = get_terms(queries.get(query_id))

print(len(doc_terms))
print(doc_terms.get("1"))
print(len(doc_terms.get("1")))

print(len(query_terms))
print(query_terms.get("1"))
print(len(query_terms.get("1")))

1460
{'18': 1, 'edit': 4, 'dewey': 3, 'decim': 2, 'class': 2, 'comarom': 1, 'j.p.': 1, 'pres': 1, 'study': 1, 'hist': 2, 'first': 2, 'ddc': 2, 'publ': 1, '1876': 1, 'eighteen': 1, '1971': 1, 'fut': 1, 'continu': 1, 'appear': 1, 'nee': 1, 'spit': 1, "'s": 1, 'long': 1, 'healthy': 1, 'lif': 1, 'howev': 1, 'ful': 1, 'story': 1, 'nev': 1, 'told': 1, 'biograph': 1, 'brief': 1, 'describ': 1, 'system': 1, 'attempt': 1, 'provid': 1, 'detail': 1, 'work': 1, 'spur': 1, 'grow': 1, 'libr': 1, 'country': 1, 'abroad': 1}
43
112
{'problem': 1, 'concern': 1, 'mak': 1, 'describ': 1, 'titl': 3, 'difficul': 1, 'involv': 1, 'autom': 1, 'retriev': 1, 'artic': 2, 'approxim': 1, 'us': 1, 'relev': 1, 'cont': 1}
14


Now, let’s represent all queries and all documents in the same shared space: for example, Table represents one query and two documents in a space where the columns of the table (dimensions in the Python data structure) are shared among all three. For instance, column 1 (first dimension) keeps the counts of the term “much” across the query and both documents, column 2 (second dimension) keeps the counts for “information”, and so on.

Now let’s add all terms from the data set as columns, and keep the counts for each of them in each query and each document as rows. In terms of Python data structures, this means that each document and each query will keep the whole dictionary of terms in the collection with the associated term frequencies.

In [23]:
# First, collect the shared vocabulary of terms used in documents and queries; return it as a sorted list for convenience
def collect_vocabulary():
  all_terms = []
  for doc_id in doc_terms.keys():
    for term in doc_terms.get(doc_id).keys():
      all_terms.append(term)
  for query_id in query_terms.keys():
    for term in query_terms.get(query_id).keys():
      all_terms.append(term)
  
  return sorted(set(all_terms))

In [24]:
all_terms = collect_vocabulary()

# the length of the shared vocabulary (you should end up with 8881 terms in total), and check the first several terms in the vocabulary
print(len(all_terms))
print(all_terms[:10])

8874
["''", "'60", "'70", "'a", "'anyhow", "'apparent", "'b", "'basic", "'better", "'bibliograph"]


In [25]:
def vectorize(input_features, vocabulary):
  output = {}
  for item_id in input_features.keys():
    features = input_features.get(item_id)
    """
    Now each query and each document can be represented with a dictionary with the same set of keys – the
    terms from the shared vocabulary. The values will either be equal to the term frequency in the particular
    query and document, or will be 0 if the term is not in the query or document
    """
    output_vector = []
    for word in vocabulary:
      if word in features.keys():
        output_vector.append(int(features.get(word)))
      else:
        output_vector.append(0)
    output[item_id] = output_vector
  
  return output

In [26]:
# Using the vectorize method you can represent all queries and documents in this shared space
doc_vectors = vectorize(doc_terms, all_terms)
query_vectors = vectorize(query_terms, all_terms)

# let's check some statistics on these data structures: you should still have 1460 doc_vectors and 112 query_vectors, with 8881 terms each
print(len(doc_vectors))
print(len(doc_vectors.get("1460")))

print(len(query_vectors))
print(len(query_vectors.get("112")))

1460
8874
112
8874


Now, another way to think about each of these term dictionaries associated with each document and each query is as vectors: that is, each document and each query is represented as a vector in a shared space, with the number of dimensions equal to the length of the shared vocabulary (8881) and the term frequencies in each dimension representing the coordinates.It reinterprets the query and two documents from Table 3.6 as vectors in two dimensions associated with terms “system” and “cost” (but you can imagine how these vectors are extended to other dimensions, too):

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/19.png?raw=1' width='800'/>

Now you can estimate the relevance, or similarity, of the query and documents using the distance between them in the vector space. But before you do that, there is one more observation due.

###Weighing words with inverse document frequency

In a collection of documents you are working with, some terms are much more frequently used across all documents than others. 

For instance, since this is a collection of articles on information science and information retrieval systems, such terms as `information` or `system` may occur in many documents while other terms like `cost` may occur in fewer
documents. Which ones are more helpful in search then? 

Imagine that you were to find the relevant documents for the query 15, `How much do information retrieval and dissemination systems cost?` If `information` and `system` occur in lots of documents, then you better focus your attention on those documents that contain other terms from the query, e.g., `dissemination` and `cost`, because it is those documents that contain these words that are more relevant. 

In other words, you would like to give these rarer terms like `dissemination`
and `cost` higher weight so that the search algorithm knows it should trust their vote for relevance more. The most straightforward way to assign such weights to the terms is to make it proportionate to the number of documents where the term occurs: the higher the number of documents, the lower the discriminative power of the term, so the lower the weight should be.

Take the term `inform` as an example (this is a stem for such words as `inform` and `information`). It occurs in 651 out of 1460, so its document frequency (df) equals `651/1460~0.45`. 

On the other hand, the term `dissemin` (stem of `dissemination`) only
occurs in 68 documents, so its `df=68/1460~0.05`. `Dissemin` is a more valuable term for the search algorithm because it is rare: if a query contains it, the documents that also contain it should be given preference. 

To assign a higher weight to `dissemin` than to `inform`, let’s take the inverse document frequency (idf): 

```
idf("inform")=1/0.45~2.22,
idf("dissemin")=1/0.05~20
```

These weights show that the rare term "dissemin" is almost 10 times more important than the much more frequent term "inform". 

There are two more things to take into account here:

- First, some terms from the shared vocabulary may not occur in any of the documents, so their df will be 0. To avoid division by 0, it is common to smooth the counts: to calculate the idf, take `(df+1)` rather than df, i.e. `idf = 1/(df+1)`, so you will never have to divide by zero, and the absolute values of idf won’t change much.
- Second, it is common to “tone down” the differences in absolute counts, as the
difference between very rare and very frequent terms might be huge, especially in large collections. It is assumed that the weight given to the terms should increase not linearly (i.e., by one with each document) but rather sub-linearly (i.e., more slowly). Logarithmic function achieves this effect: the relative order of the term’s importance doesn’t change, while the absolute number does.

To put all the components together, here are the idf values for the terms “inform” and “dissemin” in this collection:

```
idf("inform") = log10(1460/(651+1)) ~ 0.35
idf("dissemin") = log10(1460/68+1)) ~ 1.33
```

As you can see, the difference is still significant, but the counts are more comparable. The general formula then is:


$$ idf(term) = log_{10}(\frac{N} {df(term) + 1}) $$

where $N$ is the total number of documents in the collection.


Now, if a particular document contains 2 occurrences of the term “cost”, its idf-weighted value will be `2*1.02=2.04`, while if it contains 2 occurrences of the term “system”, its idfweighted value will be `2*0.44=0.88`, so despite the same term frequencies the more informative term “cost” will get higher overall weight.

For instance, here is how idf weighting will change the weights of the terms in the documents.

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/20.png?raw=1' width='800'/>


In [27]:
# Estimate idf values for each term in the vocabulary by counting how many documents contain it
def calculate_idfs(vocabulary, doc_features):
  doc_idfs = {}
  for term in vocabulary:
    doc_count = 0
    for doc_id in doc_terms.keys():
      terms = doc_terms.get(doc_id)
      if term in terms.keys():
        doc_count += 1
    doc_idfs[term] = math.log(float(len(doc_terms.keys())) / float(1 + doc_count), 10)  # Apply the idf formula
  return doc_idfs

In [28]:
doc_idfs = calculate_idfs(all_terms, doc_terms)
print(len(doc_idfs))
# Check out the results: you should have idf values for all 8881 terms from the vocabulary; the idf for any particular term should coincide with your estimates
print(doc_idfs.get("system"))

8874
0.43844122348938885


In [29]:
# Define a method to apply idf weighing to the input_terms (in particular, to doc_terms) data structure
def vectorize_idf(input_terms, input_idfs, vocabulary):
  output = {}
  for item_id in input_terms.keys():
    terms = input_terms.get(item_id)
    output_vector = []
    for term in vocabulary:
      # For that, multiply the term frequencies with the idf weights if the term is present in the document, otherwise its term frequency stays 0
      if term in terms.keys():
        output_vector.append(input_idfs.get(term) * float(terms.get(term)))
      else:
        output_vector.append(float(0))
    output[item_id] = output_vector
  return output

In [30]:
# Apply idf weighting to doc_terms
doc_vectors = vectorize_idf(doc_terms, doc_idfs, all_terms)

# Print out some statistics: the dimensionality of the data structure should still be 1460 documents by 8881 terms
print(len(doc_vectors))
print(len(doc_vectors.get("1460")))

1460
8874


Let’s now summarize what you have implemented so far:

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/21.png?raw=1' width='800'/>

##Step 4: Practical use of the search algorithm

Now that the documents and queries are represented in the shared search space, it’s time to run the search algorithm, find the most relevant documents for each query and evaluate the results.

###Retrieval of the most similar documents

How can you estimate query to document similarity based on the vector representations? 

The similarity can be interpreted as distance in space defined by the query and document vectors.
- Each document and each query are represented as vectors in a shared space, with the dimensions representing terms and coordinates representing weighted term counts
- Similarity is estimated using distances in this shared space. To eliminate the effect of different lengths, it is more reliable to use the cosine of the angle between the vectors
- The higher the cosine, the more similar the query and the document are

The cosine can be estimated using the formula:

```
cosine(vec1,vec2) = dot_product(vec1,vec2)/(length(vec1)*length(vec2))
```


In [31]:
query = [1, 1]
document = [3, 5]

def length(vector):
  sq_length = 0
  for idx in range(0, len(vector)):
    sq_length += math.pow(vector[idx], 2)
  return math.sqrt(sq_length)

In [32]:
def dot_product(vector1, vector2):
  if len(vector1) == len(vector2):
    dot_prod = 0
    for idx in range(0, len(vector1)):
      if not vector1[idx] == 0 and not vector2[idx] == 0:
        dot_prod += vector1[idx] * vector2[idx]
    return dot_prod
  else:
    return "Unmatching dimensionality"

def calculate_cosine(query, document):
  cosine = dot_product(query, document) / (length(query) * length(document))
  return cosine

In [33]:
cosine = calculate_cosine(query, document)
print(cosine)

0.9701425001453319


Let’s calculate the cosine between the query and documents `doc_x` and `doc_y`
(using only `tf` and ignoring the `idf` weighing for the sake of simplicity here):

```
cosine(query,doc_x) = (0+2+1+2+3+1)/(sqrt(6)*sqrt(19)) ~ 0.84
cosine(query,doc_y) = (0+1+1+1+2+0)/(sqrt(6)*sqrt(7)) ~ 0.77
```

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/19.png?raw=1' width='800'/>

Based on these results, `doc_x` is more similar to the query than `doc_y`, so if you apply the cosine similarity estimation for the given query to the set of two documents, you should return them ordered as `(doc_x, doc_y)`. As it is `doc_x` that is more similar and thus more relevant to the query, if you want more relevant information you should start with `doc_x`.



In [34]:
query = [1, 1]
doc_x = [3, 1]
doc_y = [2, 0]

cosine_x = calculate_cosine(query, doc_x)
cosine_y = calculate_cosine(query, doc_y)
print(cosine_x)
print(cosine_y)

0.8944271909999159
0.7071067811865475


Let’s apply cosine similarity to the input queries and documents in the dataset and return the resulting lists of relevant documents ordered by their relevance scores, that is cosine similarity values:

In [35]:
document = doc_vectors.get("27")
query = doc_vectors.get("15")

cosine = calculate_cosine(query, document) / (length(query) * length(document))
print(cosine)

0.00012969661671903527


In [36]:
document = doc_vectors.get("60")
query = doc_vectors.get("3")

cosine = calculate_cosine(query, document) / (length(query) * length(document))
print(cosine)

0.00018813808718247965


Let's apply the search algorithm to find relevant documents for a particular query:

In [37]:
results = {}

# For each document in the set of documents, calculate cosine similarity between the input query and the document
for doc_id in doc_vectors.keys():
  document = doc_vectors.get(doc_id)
  cosine = calculate_cosine(query, document)
  results[doc_id] = cosine

# Sort the results dictionary by cosine values (key=itemgetter(1)) in descending order starting with the highest value (reverse=True).
for items in sorted(results.items(), key=itemgetter(1), reverse=True)[:44]:
  print(items[0])

3
1316
232
335
1172
1345
172
142
993
1057
231
30
1214
874
412
177
1314
409
712
1227
758
898
1104
64
1253
538
1441
252
1048
804
1187
1433
1428
176
1444
403
1084
1447
160
1157
378
952
620
1350


This piece of code returns a list of 44 documents identified by the search algorithm as relevant to query 3, ordered by cosine similarity starting with the most relevant one. A quick glance over the first 10 returned documents (that is how many you would see on the first page in the Internet browser) shows that 8 out of 10 documents are also included in the gold standard. Perhaps even more importantly the top 2 documents in the returned list are
relevant according to the gold standard – and you might not even need to look any further than the first couple of documents!

### Evaluate the results

Suppose you are looking for the documents related to query 3, “What is information science? Give definitions where possible.” According to the gold standard, there are 44 documents in this set that match this query.

In some situations, you might be interested in exhaustive search, that is, you will measure the success of your algorithm by its ability to find all 44 documents. However, in most situations what you would like is for the algorithm to return the relevant documents at the top of the list: it is more
important that the first document returned by the algorithm is relevant than whether the 44th document is relevant.

Since the number of relevant documents in the gold standard varies for different queries – for example, it is 44 for query 3 but there is only 1 relevant document for query 6 – you may prefer to set the number of top documents to be returned by your algorithm in advance.

In addition, it is rarely the case that users are interested in documents after the first several relevant ones, so returning something between top 3 to top 10 documents would be reasonable.

The number of documents that are returned by the algorithm among those top-3
(top-10) that are also included as relevant in the gold standard is called true positives – they are truly relevant documents actually identified by your algorithm. The proportion of true positives to the total number of documents returned by the algorithm is called precision, and if you predefine the number of returned documents to be k this measure is called `precision@k` (e.g., `precision@3` or `precision@10`).

```
precision@10 = (true positives among the top 10 documents) / 10 =
(number of documents that are actually relevant among the top 10) / 10
```

And in the general case, precision@k is:

```
precision@k = (true positives among the top k documents) / k =
(number of documents that are actually relevant among the top k) / k
```

The higher the precision, the better the algorithm you have built, however the results may also depend on the quality of the dataset and the queries themselves.

If you want the results to be more objective, it is useful to evaluate precision across all queries. This is called mean precision, because it takes the mean across all queries.

For example, if the top-3 results for the first query are all relevant, `precision@3=1`; if only 2 are relevant, `precision@3=0.66`; for only one relevant result, `precision@3=0.33`. If you estimate the mean precision across 3 queries with such results, it would be equal to `0.66`.

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/22.png?raw=1' width='800'/>

Thus, the mean precision@k can be estimated as:

```
Mean_p@k = sum_over_queries(p@k)/number_of_queries =
sum_over_queries(true_positives/k)/number_of_queries
```

You might also be interested in knowing how often the top results contain at least one relevant document: in the case exemplified, the user will be able to find at least one relevant document in the top-3 results, which is quite useful, therefore this ratio will be equal to 1.

In [44]:
# Prefilter – only keep the documents that contain at least one word from the query – to speed search up:
def prefilter(doc_terms, query):
  docs = []
  for doc_id in doc_terms.keys():
    found = False
    i = 0
    while i<len(query.keys()) and not found:
      term = list(query.keys())[i]
      if term in doc_terms.get(doc_id).keys():
        docs.append(doc_id)
        found=True
      else:
        i+=1
  return docs

docs = prefilter(doc_terms, query_terms.get("6"))
print(docs[:100])
print(len(docs))

prefiltered_docs = {}
for query_id in mappings.keys():
  prefiltered_docs[query_id] = prefilter(doc_terms, query_terms.get(str(query_id)))

['5', '6', '10', '15', '16', '17', '21', '22', '25', '26', '27', '29', '30', '33', '38', '41', '42', '43', '45', '46', '47', '49', '51', '52', '56', '57', '58', '63', '64', '66', '68', '71', '74', '77', '78', '79', '80', '82', '87', '90', '91', '92', '95', '96', '97', '98', '101', '102', '104', '106', '107', '109', '114', '116', '117', '122', '123', '124', '126', '129', '131', '132', '136', '140', '141', '142', '144', '150', '151', '155', '157', '158', '159', '160', '163', '168', '169', '175', '178', '179', '180', '181', '191', '194', '197', '206', '208', '211', '212', '214', '218', '220', '228', '229', '233', '237', '240', '241', '242', '243']
593


In [40]:
def calculate_precision(model_output, gold_standard):
  true_pos = 0
  for item in model_output:
    if item in gold_standard:
      true_pos += 1
  # Precision equals to the number of relevant documents from the gold standard that are also returned in the top-k results by the algorithm
  return float(true_pos) / float(len(model_output))


def calculate_found(model_output, gold_standard):
  found = 0
  for item in model_output:
    if item in gold_standard:
      found = 1
  # Alternatively, give the algorithm some credit if at least one document in the top k is relevant
  return float(found)

In [45]:
precision_all = 0.0
found_all = 0.0

# Calculate mean values across all queries
for query_id in mappings.keys():
  # Gold standard is the list of relevant document ids that can be extracted from the mappings data structure
  gold_standard = mappings.get(str(query_id))
  query = query_vectors.get(str(query_id))

  results = {}
  model_output = []
  for doc_id in prefiltered_docs.keys():
    document = doc_vectors.get(doc_id)
    cosine = calculate_cosine(query, document)
    results[doc_id] = cosine  # For each document, estimate its relevance to the query with cosine similarity as before
  
  # Sort the results and only consider top-k (e.g., top-3) most relevant documents
  for items in sorted(results.items(), key=itemgetter(1), reverse=True)[:3]:
    model_output.append(items[0])
  
  precision = calculate_precision(model_output, gold_standard)
  found = calculate_found(model_output, gold_standard)
  print(str(query_id) + ": " + str(precision))

  # Accumulate evaluation values across all queries; track the results by a print out message
  precision_all += precision
  found_all += found

# In the end, estimate the mean values for all queries
print(precision_all / float(len(mappings.keys())))
print(found_all / float(len(mappings.keys())))

1: 0.6666666666666666
2: 0.0
3: 0.0
4: 0.0
5: 0.0
6: 0.0
7: 0.0
8: 0.0
9: 0.0
10: 0.3333333333333333
11: 0.3333333333333333
12: 0.0
13: 0.6666666666666666
14: 0.0
15: 0.6666666666666666
16: 0.0
17: 0.0
18: 0.3333333333333333
19: 0.3333333333333333
20: 0.3333333333333333
21: 0.0
22: 0.0
23: 0.3333333333333333
24: 0.3333333333333333
25: 0.0
26: 0.6666666666666666
27: 0.3333333333333333
28: 0.0
29: 0.0
30: 1.0
31: 0.3333333333333333
32: 0.3333333333333333
33: 0.0
34: 0.6666666666666666
35: 0.3333333333333333
37: 1.0
39: 0.3333333333333333
41: 0.0
42: 0.0
43: 0.0
44: 0.0
45: 1.0
46: 0.6666666666666666
49: 0.0
50: 1.0
52: 0.0
54: 0.3333333333333333
55: 0.0
56: 0.6666666666666666
57: 0.0
58: 0.3333333333333333
61: 0.0
62: 0.3333333333333333
65: 0.0
66: 0.3333333333333333
67: 0.3333333333333333
69: 0.3333333333333333
71: 0.6666666666666666
76: 0.3333333333333333
79: 0.0
81: 0.0
82: 0.0
84: 0.3333333333333333
90: 0.6666666666666666
92: 0.0
95: 0.3333333333333333
96: 0.0
97: 0.3333333333333333


According to the results, on some queries the algorithm performs very well: e.g., a print out message `1: 1.0` shows that all 3 documents returned for query 1 are relevant, making `precision@3` for this query equal to 1. However, on other queries the algorithm does not perform that well: e.g. `6: 0.0` – as there is only one document relevant for query 6 according to the gold standard, the algorithm fails to put it within the first 3 and gets a score of 0 for this query. The mean value of `precision@3` for this algorithm is 0.39, and in 66% of
the cases the algorithm finds at least one relevant document among the top 3.

If you are only interested in the proportion of cases when the top most relevant
document identified by the algorithm is actually relevant you can calculate that modifying the code only slightly: instead of sorting all the results and then taking the top-3 it simply needs to identify and store a single best result.


Now, let's modify the code to calculate precision@1 – i.e., the mean value across the queries when the top-1 document returned by the algorithm is indeed relevant.



In [46]:
precision_all = 0.0
for query_id in mappings.keys():
  gold_standard = mappings.get(str(query_id))
  query = query_vectors.get(str(query_id))
  result = ""
  model_output = []
  max_sim = 0.0
  prefiltered_docs = prefilter(doc_terms, query_terms.get(str(query_id)))
  for doc_id in prefiltered_docs:
    document = doc_vectors.get(doc_id)
    cosine = calculate_cosine(query, document) 
    if cosine >= max_sim:
      max_sim = cosine
      result = doc_id
  model_output.append(result)
  precision = calculate_precision(model_output, gold_standard)
  print(f"{str(query_id)}: {str(precision)}")
  precision_all += precision

print(precision_all/len(mappings.keys()))

1: 1.0
2: 0.0
3: 1.0
4: 0.0
5: 0.0
6: 0.0
7: 0.0
8: 0.0
9: 0.0
10: 1.0
11: 1.0
12: 0.0
13: 1.0
14: 0.0
15: 0.0
16: 0.0
17: 0.0
18: 0.0
19: 0.0
20: 0.0
21: 0.0
22: 0.0
23: 0.0
24: 1.0
25: 0.0
26: 1.0
27: 1.0
28: 1.0
29: 1.0
30: 1.0
31: 0.0
32: 0.0
33: 0.0
34: 1.0
35: 0.0
37: 1.0
39: 1.0
41: 1.0
42: 1.0
43: 0.0
44: 0.0
45: 1.0
46: 0.0
49: 0.0
50: 1.0
52: 1.0
54: 0.0
55: 1.0
56: 1.0
57: 0.0
58: 0.0
61: 0.0
62: 1.0
65: 1.0
66: 1.0
67: 0.0
69: 1.0
71: 0.0
76: 1.0
79: 0.0
81: 1.0
82: 0.0
84: 0.0
90: 0.0
92: 1.0
95: 0.0
96: 0.0
97: 1.0
98: 1.0
99: 0.0
100: 0.0
101: 0.0
102: 1.0
104: 0.0
109: 0.0
111: 1.0
0.42105263157894735


Finally, you may wish to know how highly, on the average, the algorithm places the relevant document in its ranking. This shows how far into the list of the returned results you should typically look to find the first relevant document. The measure that allows you to evaluate that relies on the use of the highest ranking of a relevant document identified by the algorithm. Since you already sort the returned documents by their relevance scores starting with the most relevant one, position one in this list is called first rank, position two –
second rank, and so on.

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/23.png?raw=1' width='800'/>

- The first relevant documents for both query 1 and query 2 in this example are at position 1 in the ordered lists of returned documents, so their ranks are 1; for query 3, the first relevant document is found in the second position, which gives this result rank 2.
- However, returning the first relevant document at rank 1 is better than returning the first relevant document at any further position, so your measure should reflect this by assigning a higher score to the results with the rank 1. Just like with the inverse document frequency, if you take the inverse of the ranks, you will end up with exactly such measure: for both queries 1 and 2 the algorithm returns the best possible results by placing the first relevant document at position 1, so it gets a score of 1/1=1 for that; for query 3 it returns an irrelevant document in position 1 and the first relevant
document in position 2 – for that it gets only half the full score, 1/2.

To summarize, to assign a score for the results for each query take the inverse of the rank of the first relevant document in the ordered list of results – this is called reciprocal rank:

```
reciprocal rank = 1 / rank of the first relevant document in the ordered list of results
```

- Finally, as before, you want to have a comprehensive overview of the results across all queries, so you need to take a mean reciprocal rank (MRR) for the reciprocal ranks across all queries. For the example from Figure, this will equal to `(1 + 1 + 1/2) / 3 = 0.83`.

```
MRR = sum_of_reciprocal_ranks_across_queries / number_of_queries
```

The best-case scenario is when the algorithm always puts a relevant document at the top of the list, so it assigns rank 1 in all cases. If the first relevant document is always found at rank 2, the mean will equal to 1/2; for the results at rank 3, the mean will be 1/3, and so on.

The result that you get for the example from Figure, `MRR = (1+1+1/2)/3 = 0.83`, lies between 1/2 and 1 and is closer to 1. This value shows, that on the average, the ranking of the first relevant document returned by the algorithm is between 1st and 2nd rank, and is in fact more often 1st than 2nd.



In [48]:
# MRR – rank of the first relevant entry:
rank_all = 0.0
for query_id in mappings.keys():
  # extract the list of gold standard mappings for each query
  gold_standard = mappings.get(str(query_id))
  query = query_vectors.get(str(query_id))
  results = {}
  for doc_id in doc_vectors.keys():
    document = doc_vectors.get(doc_id)
    cosine = calculate_cosine(query, document)    
    results[doc_id] = cosine
  # Sort the documents returned by the algorithm in descending order starting with the most similar. The position of each document in this sorted list is called rank
  sorted_results = sorted(results.items(), key=itemgetter(1), reverse=True)
  index = 0
  found = False
  # You only need to find the first relevant document in this list, so set the flag found to False, and switch it
  # to True as soon as you encounter the first relevant document, or reach the end of the list
  while found==False:
    item = sorted_results[index]
    # Increment index (rank) with each document in the results
    index += 1
    if index==len(sorted_results):
      found = True
    # As before, the document id is the first element in the sorted tuples of (document_id, similarity score)
    if item[0] in gold_standard:
      found = True
      print(f"{str(query_id)}: {str(float(1) / float(index))}")
      # Estimate inverse of the rank
      rank_all += float(1) / float(index)

# Calculate and print out the mean value across all queries           
print(rank_all/float(len(mappings.keys())))

1: 1.0
2: 0.3333333333333333
3: 1.0
4: 0.08333333333333333
5: 0.125
6: 0.04
7: 0.05555555555555555
8: 0.03571428571428571
9: 0.5
10: 1.0
11: 1.0
12: 0.14285714285714285
13: 1.0
14: 0.011764705882352941
15: 0.2
16: 0.02857142857142857
17: 0.25
18: 0.25
19: 0.25
20: 0.5
21: 0.0625
22: 0.09090909090909091
23: 0.3333333333333333
24: 1.0
25: 0.14285714285714285
26: 1.0
27: 1.0
28: 1.0
29: 1.0
30: 1.0
31: 0.5
32: 0.3333333333333333
33: 0.07142857142857142
34: 1.0
35: 0.5
37: 1.0
39: 1.0
41: 1.0
42: 1.0
43: 0.2
44: 0.5
45: 1.0
46: 0.5
49: 0.3333333333333333
50: 1.0
52: 1.0
54: 0.3333333333333333
55: 1.0
56: 1.0
57: 0.1111111111111111
58: 0.3333333333333333
61: 0.5
62: 1.0
65: 1.0
66: 1.0
67: 0.125
69: 1.0
71: 0.25
76: 1.0
79: 0.25
81: 1.0
82: 0.5
84: 0.045454545454545456
90: 0.14285714285714285
92: 1.0
95: 0.5
96: 0.043478260869565216
97: 1.0
98: 1.0
99: 0.5
100: 0.2
101: 0.014084507042253521
102: 1.0
104: 0.2
109: 0.5
111: 1.0
0.5647694319005727


The result – mean reciprocal rank of 0.58 – printed by this piece of code suggests that, on the average, the highest rank of a relevant document identified by this search algorithm is between 1st and 2nd, i.e. you will often find the relevant results within the first pair of returned documents.


This concludes the implementation of the search algorithm, so let’s summarize what steps you have implemented:

<img src='https://github.com/rahiakela/machine-learning-algorithms/blob/main/Essential-NLP/03-information-retrieval/images/24.png?raw=1' width='800'/>

##Assignment

Apply the search algorithm to your own data. For that, you would need to read in the files one by one.

Alternatively, apply the search algorithm to a different dataset from
http://ir.dcs.gla.ac.uk/resources/test_collections/. 

Among these, the Cranfield dataset uses a similar data format to
the CISI dataset, and is also relatively small and easy to work with.